![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_BertForZeroShotClassification.ipynb)

# Import OpenVINO BertForZeroShotClassification   models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting BertForZeroShotClassification   models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for BertForZeroShotClassification   from BertForZeroShotClassification   and they have to be in `Zero Shot Classification` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 693.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.6/342.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [nlptown/NbAiLab/nb-bert-base-mnli](https://huggingface.co/NbAiLab/nb-bert-base-mnli) model from HuggingFace as an example and load it as a `OVModelForSequenceClassification`, representing an OpenVINO model.
- In addition to the OVModelForSequenceClassification model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from optimum.intel import OVModelForSequenceClassification
from transformers import AutoTokenizer

MODEL_NAME = "NbAiLab/nb-bert-base-mnli"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

ov_model.save_pretrained(EXPORT_PATH)
tokenizer.save_pretrained(EXPORT_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


('ov_models/NbAiLab/nb-bert-base-mnli/tokenizer_config.json',
 'ov_models/NbAiLab/nb-bert-base-mnli/special_tokens_map.json',
 'ov_models/NbAiLab/nb-bert-base-mnli/vocab.txt',
 'ov_models/NbAiLab/nb-bert-base-mnli/added_tokens.json',
 'ov_models/NbAiLab/nb-bert-base-mnli/tokenizer.json')

- We need to move `vocabs.txt` from the tokenizer to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [3]:
!mkdir -p {EXPORT_PATH}/assets

with open(f"{EXPORT_PATH}/assets/labels.txt", "w") as f:
    f.write('\n'.join(ov_model.config.id2label[k] for k in sorted(ov_model.config.id2label)))

!mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets/

In [4]:
!ls -l {EXPORT_PATH}/assets

total 980
-rw-r--r-- 1 root root     32 Jun 20 01:30 labels.txt
-rw-r--r-- 1 root root 995526 Jun 20 01:30 vocab.txt


- Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

## Import and Save BertForZeroShotClassification in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [5]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 23.5 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [6]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `BertForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `BertForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [7]:
from sparknlp.annotator import BertForZeroShotClassification

zero_shot_classifier = BertForZeroShotClassification.loadSavedModel(
    EXPORT_PATH,
    spark
    )\
    .setInputCols(["document", "token"]) \
    .setOutputCol("class") \
    .setCandidateLabels(["urgent", "mobile", "travel", "movie", "music", "sport", "weather", "technology"])

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [8]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp_openvino".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [9]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your BertForZeroShotClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [10]:
! ls -l {MODEL_NAME}_spark_nlp_openvino

total 695236
-rw-r--r-- 1 root root 711907715 Jun 20 01:34 bert_classification_openvino
drwxr-xr-x 4 root root      4096 Jun 20 01:33 fields
drwxr-xr-x 2 root root      4096 Jun 20 01:33 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BertForZeroShotClassification model 😊

In [11]:
zero_shot_classifier_loaded = BertForZeroShotClassification.load("./{}_spark_nlp_openvino".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [12]:
zero_shot_classifier_loaded.getClasses()

['contradiction', 'neutral', 'entailment']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
from pyspark.ml import Pipeline
from sparknlp.annotator import Tokenizer
from sparknlp.base import DocumentAssembler

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

example = spark.createDataFrame(
     [["I have a problem with my iphone that needs to be resolved asap!!"],
      ["Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."],
      ["I have a phone and I love it!"],
      ["I really want to visit Germany and I am planning to go there next year."],
      ["Let's watch some movies tonight! I am in the mood for a horror movie."],
      ["Have you watched the match yesterday? It was a great game!"],
      ["We need to harry up and get to the airport. We are going to miss our flight!"]], ["text"])

model = pipeline.fit(example)
result = model.transform(example)

result.select("text", "class.result").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------+--------+
|text                                                                                                          |result  |
+--------------------------------------------------------------------------------------------------------------+--------+
|I have a problem with my iphone that needs to be resolved asap!!                                              |[mobile]|
|Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app.|[mobile]|
|I have a phone and I love it!                                                                                 |[mobile]|
|I really want to visit Germany and I am planning to go there next year.                                       |[travel]|
|Let's watch some movies tonight! I am in the mood for a horror movie.                                         |[movie] |
|Have you watched the ma

That's it! You can now go wild and use hundreds of `BertForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀
